In [63]:
import requests
import pandas as pd
import yfinance as yf
import plotly as plt
import plotly.express as px
import streamlit as st
import scipy.stats as stats
from scipy.stats import ttest_ind, f_oneway
import numpy as np

In [64]:
tickers = [
    "WEGE3.SA", "KEPL3.SA", "ROMI3.SA", "TASA3.SA", "SHUL4.SA", "POMO4.SA",
    "VLID3.SA", "EMBR3.SA", "RAPT4.SA", "TUPY3.SA", "RAIL3.SA", "GOLL4.SA",
    "AZUL4.SA", "JSLG3.SA", "LOGN3.SA"
]

# Função para testar se os tickers estão disponíveis
def check_tickers_availability(tickers):
    valid_tickers = []
    for ticker in tickers:
        try:
            df = yf.download(ticker, period="1mo", progress=False)
            if not df.empty:
                valid_tickers.append(ticker)
        except Exception as e:
            print(f"Erro ao verificar {ticker}: {e}")
    return valid_tickers

# Verificar tickers válidos
valid_tickers = check_tickers_availability(tickers)

# Exibir os tickers que funcionam
print("Tickers disponíveis:", valid_tickers)

Tickers disponíveis: ['WEGE3.SA', 'KEPL3.SA', 'ROMI3.SA', 'TASA3.SA', 'SHUL4.SA', 'POMO4.SA', 'VLID3.SA', 'EMBR3.SA', 'RAPT4.SA', 'TUPY3.SA', 'RAIL3.SA', 'GOLL4.SA', 'AZUL4.SA', 'JSLG3.SA', 'LOGN3.SA']


In [65]:

# Lista de tickers válidos
valid_tickers = ['WEGE3.SA', 'KEPL3.SA', 'ROMI3.SA', 'TASA3.SA', 'SHUL4.SA', 
                 'POMO4.SA', 'VLID3.SA', 'EMBR3.SA', 'RAPT4.SA', 'TUPY3.SA', 
                 'RAIL3.SA', 'GOLL4.SA', 'AZUL4.SA', 'JSLG3.SA', 'LOGN3.SA']

# Definir o período de análise
start_date = "2019-01-01"
end_date = "2024-12-31"

# Função para baixar os preços de fechamento ajustados das ações
def get_stock_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date, interval='1mo', progress=False)['Close']
    return data

# Baixando os dados das ações disponíveis
stock_data = get_stock_data(valid_tickers, start_date, end_date)

# Exibir as primeiras linhas dos dados coletados
stock_data = stock_data.reset_index()

In [66]:

# Função para obter dados da SELIC via API do Banco Central do Brasil (BCB)
def get_selic_data():
    url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.4189/dados?formato=json"  # Código 4189 corresponde à SELIC
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df_selic = pd.DataFrame(data)
        df_selic['data'] = pd.to_datetime(df_selic['data'], format='%d/%m/%Y')
        df_selic['valor'] = df_selic['valor'].astype(float)
        df_selic.rename(columns={'data': 'Date', 'valor': 'Selic'}, inplace=True)
        return df_selic
    else:
        print("Erro ao obter dados da SELIC")
        return None

# Obtendo os dados da SELIC
df_selic = get_selic_data()

# Filtrar os dados para manter apenas de 2019 a 2024
df_selic_filtered = df_selic[(df_selic['Date'] >= "2019-01-01") & (df_selic['Date'] <= "2024-12-31")]

# Exibir as primeiras linhas dos dados filtrados
print(df_selic_filtered.head())



          Date  Selic
389 2019-01-01    6.4
390 2019-02-01    6.4
391 2019-03-01    6.4
392 2019-04-01    6.4
393 2019-05-01    6.4


In [67]:
df_fed_funds = pd.read_csv("FEDFUNDS.csv", parse_dates=['observation_date'])

# Renomeando colunas
df_fed_funds.rename(columns={'observation_date': 'Date', 'FEDFUNDS': 'Fed_Funds'}, inplace=True)

# Filtrando os dados entre 2019 e 2024
df_fed_funds = df_fed_funds[(df_fed_funds['Date'] >= "2019-01-01") & (df_fed_funds['Date'] <= "2024-12-31")]

# Exibir os dados carregados
print(df_fed_funds.head())


        Date  Fed_Funds
0 2019-01-01       2.40
1 2019-02-01       2.40
2 2019-03-01       2.41
3 2019-04-01       2.42
4 2019-05-01       2.39


In [68]:
# Definir índice como data para facilitar junção
df_selic_filtered.set_index('Date', inplace=True)
df_fed_funds.set_index('Date', inplace=True)

# Juntar os dados pelo índice de data
df_rates = df_selic_filtered.join(df_fed_funds, how='inner')

# Resetar o índice
df_rates.reset_index(inplace=True)

# Exibir as primeiras linhas para validar
print(df_rates.head())


        Date  Selic  Fed_Funds
0 2019-01-01    6.4       2.40
1 2019-02-01    6.4       2.40
2 2019-03-01    6.4       2.41
3 2019-04-01    6.4       2.42
4 2019-05-01    6.4       2.39


---

### **📊 Correlação entre Selic e Fed Funds Rate**


---


In [88]:
# Criar um heatmap interativo da correlação entre Selic e Fed Funds Rate
fig = px.imshow(
    df_rates[['Selic', 'Fed_Funds']].corr(),
    text_auto=True,
    color_continuous_scale=["#dbe9f6", "#08306b"],  # Dois tons de azul
    title="Correlação entre Selic e Fed Funds Rate"
)

# Ajustar o layout para aumentar o tamanho da fonte
fig.update_layout(
    title_font_size=20,  # Tamanho da fonte do título
    font=dict(size=14),  # Tamanho da fonte geral
    coloraxis_colorbar=dict(title="Correlação", title_font_size=14, tickfont_size=12)  # Fonte da barra de cores
)

# Exibir o gráfico interativo
fig.show()



---

### **📊 O QUE ESSE GRÁFICO MOSTRA?**
1. **Cores e Valores**  
   - O quadrado vermelho (1.0) mostra que **Selic tem correlação perfeita com ela mesma** (óbvio).
   - O quadrado azul com **0.737** indica que **existe uma correlação positiva forte** entre a **Selic e o Fed Funds Rate**.
   - Isso significa que, quando o **Fed Funds Rate sobe**, a **Selic tende a subir também**.

2. **Correlação de 0.737**
   - Esse valor **não é 1.0**, ou seja, **não é uma relação perfeita**, mas ainda assim é uma **forte correlação positiva**.
   - Isso indica que as taxas de juros do Brasil e dos EUA se movem **na mesma direção** na maioria das vezes, mas não sempre.

---

### **📌 INTERPRETAÇÃO PRÁTICA**
- Quando o **Fed aumenta os juros**, isso pode influenciar o Banco Central do Brasil a **também aumentar a Selic**.
- Isso acontece porque **juros mais altos nos EUA fazem os investidores retirarem dinheiro do Brasil**, forçando o BC a subir os juros para manter a atratividade do país.
- Por outro lado, se os **EUA cortam os juros**, o Brasil pode seguir essa tendência **para estimular o crescimento econômico**.



---
### **📌  O que vamos fazer? - Estatística Por Trás do Grafico📌**
1. ✅ Aplicar o Teste de Pearson para verificar se a correlação observada é estatisticamente significativa.

**✅ Hipóteses do teste:**
- H₀ (Hipótese Nula): Não há correlação significativa entre Selic e Fed Funds Rate.
- H₁ (Hipótese Alternativa): Existe uma correlação significativa entre Selic e Fed Funds Rate. ✅ Nível de Significância: 5% (0.05). Se p < 0.05, rejeitamos H₀ e aceitamos que há correlação significativa.

---

In [70]:
from scipy.stats import pearsonr

# Remover valores nulos para evitar erros
df_rates.dropna(subset=['Selic', 'Fed_Funds'], inplace=True)

# Aplicar o teste de correlação de Pearson
correlation, p_value = pearsonr(df_rates['Selic'], df_rates['Fed_Funds'])

# Exibir resultados
print(f"Correlação de Pearson: {correlation:.4f}")
print(f"Valor-p: {p_value:.6f}")

# Interpretar os resultados
if p_value < 0.05:
    print("➡️ O resultado é estatisticamente significativo. Existe uma correlação entre a Selic e o Fed Funds Rate.")
else:
    print("❌ O resultado NÃO é estatisticamente significativo. Não podemos afirmar que há correlação real entre as taxas.")


Correlação de Pearson: 0.7371
Valor-p: 0.000000
➡️ O resultado é estatisticamente significativo. Existe uma correlação entre a Selic e o Fed Funds Rate.


---
### **1️⃣ Intervalo de Confiança para a Correlação**
 - Queremos calcular um **intervalo de confiança (IC)** de **95%** para a correlação entre **Selic e Fed Funds Rate**.  
 - Isso nos ajudará a entender a **precisão** da correlação observada.
---

In [71]:
# Número de observações
n = len(df_rates.dropna(subset=['Selic', 'Fed_Funds']))

# Correlação e p-valor já calculados antes
correlation, _ = pearsonr(df_rates['Selic'], df_rates['Fed_Funds'])

# Calcular erro padrão da correlação
se = 1 / np.sqrt(n - 3)

# Calcular intervalo de confiança de 95%
z_score = np.arctanh(correlation)  # Conversão para Z-score
z_critical = stats.norm.ppf(0.975)  # Valor crítico para 95%
ci_lower = np.tanh(z_score - z_critical * se)
ci_upper = np.tanh(z_score + z_critical * se)

# Exibir resultados
print(f"Intervalo de Confiança (95%) da Correlação:")
print(f"[{ci_lower:.4f}, {ci_upper:.4f}]")

Intervalo de Confiança (95%) da Correlação:
[0.6095, 0.8275]


---
### **📊 Interpretação**
 - Como o intervalo não inclui 0, podemos concluir que a correlação é estatisticamente significativa.
 - O intervalo é relativamente estreito, indicando boa precisão na estimativa da correlação.
 - A correlação é positiva e forte (entre 0.61 e 0.83), sugerindo que quando o Fed Funds Rate sobe, a Selic tende a acompanhar.
 ---

---

## 📊 **Distribuição Anual da Selic (Boxplot)**

1. 
   - Cada caixa representa a **distribuição da taxa Selic** ao longo de cada ano.  
   - As caixas mostram o comportamento da taxa ao longo do tempo: valores mínimos, máximos, mediana e dispersão.
---

In [72]:
# ==========================
# 1️⃣  PREPARAÇÃO DOS DADOS
# ==========================
# Criar coluna com o ano para segmentação
df_rates['Ano'] = df_rates['Date'].dt.year

# Filtrar dados para garantir que só temos anos completos
anos_disponiveis = df_rates['Ano'].unique()
grupos_selic = [df_rates[df_rates['Ano'] == ano]['Selic'].dropna() for ano in anos_disponiveis]

# ==========================
# 2️⃣  TESTE ANOVA
# ==========================
stat, p_value = f_oneway(*grupos_selic)

In [73]:
# ==========================
# 3️⃣  INTERPRETAÇÃO AUTOMÁTICA
# ==========================
significancia = 0.05
if p_value < significancia:
    resultado = "A diferença entre os anos é estatisticamente significativa. Ou seja, a Selic variou de forma relevante ao longo dos anos."
else:
    resultado = "Não há evidências estatísticas suficientes para afirmar que houve mudanças significativas na Selic entre os anos."

# ==========================
# 4️⃣  VISUALIZAÇÃO INTERATIVA (com ajuste de proporção)
# ==========================
fig = px.box(df_rates, x='Ano', y='Selic', title='Distribuição da Selic ao longo dos anos')

# Ajustar proporções do gráfico para evitar que fique muito retangular
fig.update_layout(
    width=1900,  # Largura ajustada
    height=800,  # Altura reduzida para melhor proporção
    margin=dict(l=40, r=40, t=60, b=40)  # Margens ajustadas
)

# ==========================
# 5️⃣  EXIBIR RESULTADOS
# ==========================
print(f"F-Estatística: {stat:.4f}")
print(f"Valor-p: {p_value:.4f}")
print(f"Conclusão: {resultado}")

fig.show()



F-Estatística: 149.5436
Valor-p: 0.0000
Conclusão: A diferença entre os anos é estatisticamente significativa. Ou seja, a Selic variou de forma relevante ao longo dos anos.


---

## 🧪 INTERPRETAÇÃO ESTATÍSTICA (ANOVA)

Aplicamos um **Teste ANOVA** para verificar se as variações na taxa Selic entre os anos são **estatisticamente significativas**.

### ✔️ Resultado do Teste:
- **F-Estatística**: `149.5436`
- **Valor-p**: `0.0000`  
  ✅ *Significativo ao nível de 5% (p < 0.05)*

### 🔎 Conclusão:
> A diferença entre os anos é estatisticamente significativa.  
> Ou seja, a **Selic variou de forma relevante ao longo dos anos**, e essa mudança **não é fruto do acaso**.

---

## 🧠 INTERPRETAÇÃO PRÁTICA

- A Selic passou por **momentos de forte queda** (2020–2021) para estimular a economia na pandemia.  
- Posteriormente, teve uma **alta expressiva** (2022 em diante) para conter a inflação.  
- Essa variação impacta diretamente **o custo do crédito, o investimento em renda fixa, o câmbio e o desempenho da Bolsa**.

##  **Evidência Visual de Variação**
   - É possível notar visualmente que a Selic teve **comportamentos distintos** em diferentes anos.  
   - Por exemplo, em **2020 e 2021**, a taxa esteve notavelmente mais baixa e volátil.  
   - Em **2022 e 2023**, observamos uma forte elevação da taxa.
---


🔁 Relacionar a variação da Selic com o retorno médio das ações industriais.

In [74]:
# Garantir que a coluna 'Date' esteja em formato datetime
df_rates['Date'] = pd.to_datetime(df_rates['Date'])

# Calcular a variação percentual da Selic mês a mês
df_rates['Variação_Selic'] = df_rates['Selic'].pct_change() * 100


In [80]:
from scipy.stats import pearsonr
import plotly.express as px

# Calcular retorno mensal das ações (em %)
returns = stock_data.set_index('Date').pct_change().dropna() * 100

# Calcular variação percentual da Selic
df_rates['Variação_Selic'] = df_rates['Selic'].pct_change() * 100
selic_var = df_rates[['Date', 'Variação_Selic']].set_index('Date')

# Juntar retornos com variação da Selic
combined = returns.join(selic_var, how='inner')

# Calcular correlação de cada ação com a variação da Selic
correlacoes = {}
for ticker in returns.columns:
    correlacao, _ = pearsonr(combined[ticker], combined['Variação_Selic'])
    correlacoes[ticker] = correlacao

# Selecionar as 5 ações mais correlacionadas com a Selic
top_5_tickers = pd.Series(correlacoes).sort_values(ascending=False).head(15).index.tolist()

# Criar DataFrame com preços dessas ações + Selic + Fed Funds
df_top5 = stock_data[['Date'] + top_5_tickers].copy()
df_taxas = df_rates[['Date', 'Selic', 'Fed_Funds']]
df_plot = pd.merge(df_top5, df_taxas, on='Date', how='inner')

# Reestruturar para gráfico interativo
df_long = df_plot.melt(id_vars='Date', var_name='Variável', value_name='Valor')

# Gráfico interativo
fig = px.line(df_long, x='Date', y='Valor', color='Variável',
              title='Evolução de Preço das Ações mais Correlacionadas com Selic vs Selic e Fed Funds',
              labels={'Valor': 'Valor (%)', 'Date': 'Data'})
# Criar DataFrame de correlação com ranking
df_correlacao = pd.DataFrame.from_dict(correlacoes, orient='index', columns=['Correlação_Selic'])
df_correlacao = df_correlacao.sort_values(by='Correlação_Selic', ascending=False).reset_index()
df_correlacao.columns = ['Ticker', 'Correlação_Selic']
df_correlacao['Ranking'] = df_correlacao['Correlação_Selic'].rank(ascending=False).astype(int)

# Visualizar o ranking
import plotly.express as px
fig_corr = px.bar(df_correlacao, x='Ticker', y='Correlação_Selic',
                  title='Ranking de Correlação com a Variação da Selic (%)',
                  text='Ranking',
                  color='Correlação_Selic',
                  color_continuous_scale='RdBu')

fig_corr.update_traces(textposition='outside')
fig_corr.update_layout(height=900, width=1900)
fig_corr.show()

# Se quiser apenas o DataFrame tabular também:
df_correlacao

#fig.update_layout(height=500, width=1000)
#fig.show()


,Ticker,Correlação_Selic,Ranking
0,LOGN3.SA,0.151269,1
1,SHUL4.SA,0.145172,2
2,KEPL3.SA,0.083113,3
3,GOLL4.SA,0.080654,4
4,RAIL3.SA,0.051265,5
5,VLID3.SA,0.037218,6
6,TASA3.SA,0.013353,7
7,TUPY3.SA,0.002413,8
8,EMBR3.SA,-0.032349,9
9,AZUL4.SA,-0.073561,10


In [81]:
import plotly.express as px

# Reestruturar para gráfico interativo
df_long = df_plot.melt(id_vars=['Date'], var_name='Variável', value_name='Valor')

# Gráfico com cores visíveis e interativas
fig = px.line(
    df_long,
    x='Date',
    y='Valor',
    color='Variável',
    color_discrete_sequence=px.colors.qualitative.Plotly,  # Cores diferentes!
    title='Evolução de Preço das Ações mais Correlacionadas com Selic e Fed Funds',
    labels={
        'Valor': 'Valor (%)',
        'Date': 'Data'
    }
)

# Layout ajustado
fig.update_layout(
    height=800,
    width=1900,
    legend_title_text='Selecione variáveis para visualizar',
    hovermode='x unified'
)

fig.show()

